<a href="https://colab.research.google.com/github/ksksingh022/bank_poc/blob/main/Bank_poc_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph Setup Guide

Follow the steps below to set up your Neo4j instance and generate a Knowledge Graph using your policy documents.

## Prerequisites

- A Neo4j Aura account (create one if you don’t have it already).
- Access to the [Neo4j AuraDB](https://neo4j.com/product/auradb/) tool.
- A HuggingFace account and API token (to download the embedding model).

## Step-by-Step Instructions

### 1. Create a Neo4j Instance
- Log in to your Neo4j Aura account.
- Set up a new Neo4j database instance.

### 2. Connect Your Neo4j Database
- Visit the [LLM Graph Builder](https://llm-graph-builder.neo4jlabs.com/).
- Connect your newly created Neo4j database using your Neo4j credentials.

### 3. Upload and Generate Knowledge Graph
- Upload your policy document in the LLM Graph Builder.
- Follow the prompts to generate a Knowledge Graph based on the document.

### 4. Obtain a HuggingFace Token
- Log in to your HuggingFace account (or create one if needed).
- Navigate to your account settings and generate an API token.
- This token will be required to download the embedding model from HuggingFace.

## Resources
- Sample documents and CSV data (50 rows): [Google Drive Link](https://drive.google.com/drive/folders/17Nj4AAtdNXP4QAkeau2RCtNdFJt8HehA).

> **Note:** Make sure you have all necessary access permissions for the Google Drive link.

With these steps, you’ll have your Neo4j database and Knowledge Graph up and running!


# Installations and Imports

In [ ]:
!pip --quiet install instructor
!pip -q install openai
!pip install -q sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
import instructor
from pydantic import BaseModel, Field
import json
from openai import AzureOpenAI
from enum import Enum
from typing import List, Literal
from instructor import llm_validator
from sqlalchemy import create_engine, text
import pandas as pd



api_key='<your api key>'
api_version="<you-api-version>"
api_base="<you-api-endpoint>"
az_client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=f"{api_base}"
)

# Database Initialization

In [ ]:
df1 = pd.read_excel("/content/credit_risk_sample.xlsx",header=0).fillna(value = 0)
df2 = pd.read_excel("/content/customer_detail_sample.xlsx",header=0).fillna(value = 0)
df3 = pd.read_excel("/content/transaction_sample.xlsx",header=0).fillna(value = 0)
database_file_path = "/content/test.db"
engine = create_engine(f'sqlite:///{database_file_path}')

df1.to_sql(
    'LoanTable',
    con=engine,
    if_exists='replace',
    index=False
)
df2.to_sql(
    'Customer',
    con=engine,
    if_exists='replace',
    index=False
)
df3.to_sql(
    'Transaction',
    con=engine,
    if_exists='replace',
    index=False
)

# Core Implementation

In [ ]:
class Query(BaseModel):
    """Class representing a single question in a decomposed from a complex question"""

    id: int = Field(..., description="Unique id of the query")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    # dependencies: List[int] = Field(
    #     default_factory=list,
    #     description="List of sub questions that need to be answered before asking this question",
    # )
    data_src: Literal["DATABASE","POLICY_DOCS"] = Field(
        description="""Source of data that should be used to answer the question.""",
    )
    # table_name: Literal["LoanTable","Customer","Transaction"] = Field(
    #     description="This is valid only If data source is DATABASE, it represents name of the table that should be used to answer the question",
    # )
    reasoning: str = Field(
        description="Reason and justify why the selected data source is suitable for answering the question",
    )
    column: List[str] = Field(
        description="This is valid only If data source is DATABASE, it represnets list of columns from corresponding table that should be used to answer the question",
    )
    sql_query: str = Field(
        description="This is valid only If data source is DATABASE, it represents SQL query to fetch dta from table."
    )
    confidence: float = Field(
        ge=0.0,
        le=1.0,
        description="Probability of correct data_src prediction"
    )

In [ ]:
class QueryList(BaseModel):
    """Class representing a list of sub-questions decomposed from a complex question"""

    queries: List[Query] = Field(
        ...,
        description="List of sub questions that need to be answered before asking the main question",
    )

In [ ]:
db_schema = """
DATABASE NAME: BankingDB

TABLE: Transaction
Description: Stores all customer transactions including deposits, withdrawals, and transfers.
Columns:
1. TransactionID (VARCHAR, Primary Key, Auto Increment) - Unique identifier for each transaction.
2. CustomerID (VARCHAR, Foreign Key) - Identifier linking the transaction to a customer in the customer_detail table.
3. CustomerAccountBalance (DECIMAL, Not Null) - Account balance after the transaction.
4. TransactionDate (DATE, Not Null) - Date when the transaction occurred.
5. TransactionTime (TIME, Not Null) - Time when the transaction occurred.
6. TransactionAmount (DECIMAL, Not Null) - Amount involved in the transaction.

TABLE: Customer
Description: Stores personal details of each customer.
Columns:
1. CustomerID (VARCHAR, Primary Key) - Unique identifier for each customer.
2. CustomerDOB (DATE, Not Null) - Date of birth of the customer.
3. CustomerGender (VARCHAR(10), Not Null) - Gender of the customer (e.g., Male, Female).
4. CustomerLocation (VARCHAR(100), Not Null) - Residential location of the customer.

TABLE: LoanTable
Description: Contains customer credit risk information and loan details.
Columns:
1. CustomerID (VARCHAR, Foreign Key) - Identifier linking the credit risk information to a customer in the customer_detail table.
2. CustomerIncome (DECIMAL, Not Null) - Annual income of the customer.
3. CustomerHomeOwnership (VARCHAR(20), Not Null) - Home ownership status (e.g., Own, Rent).
4. CustomerEmpLength (INT, Not Null) - Length of employment in years.
5. LoanIntent (VARCHAR(50), Not Null) - Purpose of the loan (e.g., Personal, Home, Auto).
6. LoanGrade (VARCHAR(5), Not Null) - Loan risk grade (e.g., A, B, C).
7. LoanAmount (DECIMAL, Not Null) - Total loan amount approved for the customer.
8. LoanInterestRate (DECIMAL, Not Null) - Interest rate applicable on the loan.
9. LoanStatus (VARCHAR(20), Not Null) - Current status of the loan (e.g., Active, Closed).
10. LoanPercentIncome (DECIMAL, Not Null) - Percentage of income allocated for loan repayment.
11. CbCustomerDefaultOnFile (BOOLEAN, Not Null) - Indicator if the customer has any defaults on file.
12. CbCustomerCredHistLength (INT, Not Null) - Length of the customer’s credit history in years.
"""

In [ ]:
SYSTEM_PROMPT="""
You are given two types of input:
1. A set of PDF documents that provide descriptive information, definitions, and context.
2. A database schema that describes the structure of a database, including tables, fields, relationships, and constraints.
Your task is to:
1. Analyze a given user query and database schema you have to break the query into sub-queries, determining which sub-query requires information from the PDF documents and which sub-query requires data from the database schema.
2. If the query matches the database schema, provide the sql query to fetch the same data from the database.
3. To write SQL query you might need to access multiple table then you should write a query with join operation accessing columns from multiple tables.
"""

In [ ]:
client = instructor.from_openai(az_client)
# question = """"""
def getOutput(question:str)-> Query:
  messages = [
          {
              "role": "system",
              "content": f"{SYSTEM_PROMPT}",
          },
          {
              "role": "user",
              "content": f"Question: {question}\ndb_schema:{db_schema}\nGenerate the sub queries.",
          },
      ]
  root = client.chat.completions.create(
          model="ak-gpt-4o-mini",
          temperature=0,
          response_model=QueryList,
          messages=messages
      )
  return root

In [ ]:
question = "Am I eligible for a home loan refinancing, and what would be the new interest rate based on my credit history and the bank's loan policies? What documentation will I need to provide? My CustomerID is C1010035."

In [ ]:
output = getOutput(question)
output.model_dump()

{'queries': [{'id': 1,
   'question': 'Am I eligible for home loan refinancing?',
   'data_src': 'DATABASE',
   'reasoning': 'Eligibility for refinancing typically depends on credit history, income, and current loan status, which can be found in the LoanTable.',
   'column': ['CustomerID',
    'LoanStatus',
    'LoanGrade',
    'CustomerIncome',
    'CbCustomerDefaultOnFile'],
   'sql_query': "SELECT CustomerID, LoanStatus, LoanGrade, CustomerIncome, CbCustomerDefaultOnFile FROM LoanTable WHERE CustomerID = 'C1010035';",
   'confidence': 0.9},
  {'id': 2,
   'question': 'What would be the new interest rate based on my credit history?',
   'data_src': 'DATABASE',
   'reasoning': "The new interest rate is determined by the customer's credit history and loan grade, which are available in the LoanTable.",
   'column': ['LoanInterestRate', 'LoanGrade'],
   'sql_query': "SELECT LoanInterestRate, LoanGrade FROM LoanTable WHERE CustomerID = 'C1010035';",
   'confidence': 0.9},
  {'id': 3,
   '

In [ ]:
queries = output.model_dump()['queries']

## Fetching Context from DB and RA
###These are two seperate processes and can be ran parallely

### 1. DB Context

In [ ]:
database_file_path = "/content/test.db"
engine = create_engine(f'sqlite:///{database_file_path}')

In [ ]:
def runQuery(sql_query,sql_engine):
    with sql_engine.connect() as connection:
      result = connection.execute(text(sql_query))
    keys = result.keys()
    values = result.fetchall()
    return [{key: value for key, value in zip(keys, value)} for value in values]

In [ ]:
db_context = ""
for subquery in queries:
  if subquery['data_src'] == 'DATABASE':
    resp = runQuery(subquery['sql_query'],engine)
    for d in resp:
      db_context += str(d)

In [ ]:
print(db_context)

{'CustomerID': 'C1010035', 'LoanStatus': 1, 'LoanGrade': 'E', 'CustomerIncome': 76000, 'CbCustomerDefaultOnFile': 'Y'}{'LoanInterestRate': 16.35, 'LoanGrade': 'E'}


### 2. Policy Documents Context

In [ ]:
import os
os.environ["NEO4J_URI"] = "<your-neo4j-uri>"
os.environ["NEO4J_USERNAME"] = "<your-neo4j-username>"
os.environ["NEO4J_PASSWORD"] = "<your-neo4j-password>"
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

In [ ]:
!pip install -q sentence-transformers
!pip install -q langchain_community
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#You will be requirest Hugging Face Token for running this. Make sure to get one from Hugging Face.
embeddings_384 = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2"#, cache_folder="/embedding_model"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00


<ipython-input-27-31ed6c3258a2>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_384 = SentenceTransformerEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install --quiet neo4j
from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(embeddings_384,node_label="Document",embedding_node_property="embedding",text_node_properties=["text"])
vector_retriever = vector_index.as_retriever()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
policy_docs_context = ""
for subquery in queries:
  if subquery['data_src'] == 'POLICY_DOCS':
    docs = vector_retriever.get_relevant_documents(subquery['question'])
    for d in docs:
      policy_docs_context += d.page_content[7:] + "\n"
policy_docs_context

<ipython-input-29-4649b0ee7609>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vector_retriever.get_relevant_documents(subquery['question'])


'- Income: The applicant must have a minimum monthly income of 25,000 for personal loans and 40,000 for business loans. Home loan applicants must provide proof of stable income. - Employment: Applicants should be employed for a minimum of 2 years, or self-employed for at least 3 years. - Credit Score: A minimum credit score of 650 is required for all loans. Customers with a higher credit score may qualify for lower interest rates. 4. Documentation Requirements: Applicants are required to submit the following documents: - Proof of Identity: A government-issued ID, such as an Aadhaar card, PAN card, or passport. - Proof of Income: Salary slips for the last 3 months, or IT returns for the last 2 years for self-employed individuals. - Bank Statements: Bank account statements for the last 6 months. - Collateral Documents: For secured loans such as home and auto loans, proof of ownership or purchase agreement is required. 5. Interest Rates: Interest rates vary\n, and all terms are communicat

## Output By combining contexts

In [ ]:
class Answer(BaseModel):
    """Class representing a Answer to the original question"""
    question: str = Field(
        ...,
        description="Original question asked using a question answering system",
    )

    answer: str = Field(
        ...,
        description="Answer to the original question based on provided context and user data",
    )
    confidence: float = Field(
        ge=0.0,
        le=1.0,
        description="Probability of correct data_src prediction"
    )
    additional_info: str = Field(
        description="Additional info (if any) required which can make the answer more accurate",
    )

In [ ]:
def getCombinedAnswer(query,client,context,user_data):

  messages = [
        {
            "role": "system",
            "content": """
            - Given the Query,UserData and Context you have to **GET DATA RELATED TO USER FROM UserData AND
              USE Context AS SOME KIND OF RULE DATA, BASED ON USER DATA AND THIS RULE DATA ANSWER THE QUERY**.
            - If there are multiple rules applicable to answer the Query, then you must restur **ALL APPLICABLE RULES.**
            - First **ANSWER WHATEVER YOU CAN ANSWER FROM CONTEXT AND USERDATA** Then If you think some additional
            information is required to answer this Query then you should **LIST DOWN** what additional is required.""",
        },
        {
            "role": "user",
            "content": f"Context:{context}\nUserData:{user_data}\nQuery:{query}\n.",
        },
    ]
  answer = client.chat.completions.create(
        model="<your-azure-model-deplyment-name>",
        temperature=0,
        response_model=Answer,
        messages=messages
    )

  return answer.model_dump()

In [ ]:
from IPython.display import HTML
import pandas as pd

def dict_to_html(data_dict):
    # Convert the dictionary to a pandas DataFrame for better HTML formatting
    df = pd.DataFrame([data_dict])
    # Generate HTML table from the DataFrame
    html_table = df.to_html(index=False, escape=False)
    return html_table

combined_answer = getCombinedAnswer(query=question,client=client,context=policy_docs_context,user_data=db_context)

In [ ]:
def dict_to_custom_html(data_dict):
    html_content = "<html><body>"
    for key, value in data_dict.items():
        # Convert the key into a heading
        html_content += f"<h3>{key.replace('_', ' ').title()}</h3>"
        # Replace newline characters with HTML line breaks
        formatted_value = value.replace("\n", "<br>") if isinstance(value, str) else value
        html_content += f"<p>{formatted_value}</p>"
    html_content += "</body></html>"
    return html_content
# Convert the dictionary to HTML
html_content = dict_to_custom_html(combined_answer)

# Render the HTML in Google Colab
HTML(html_content)